In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

spark = (SparkSession.builder.appName("x")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider",
                 "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
         .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")

In [4]:
schema = (StructType()
          .add("isin", "string")
          .add("mnemonic", "string")
          .add("securityDesc", "string")
          .add("securityType", "string")
          .add("currency", "string")
          .add("securityID", "long")
          .add("date", "date")
          .add("time", "string")
          .add("startPrice", "float")
          .add("maxPrice", "float")
          .add("minPrice", "float")
          .add("endPrice", "float")
          .add("tradedVolume", "float")
          .add("numberOfTrades", "integer"))

df = (spark
      .read
      .csv("s3a://deutsche-boerse-xetra-pds/2022-02*/", schema=schema, header=True))
df.show()

+------------+--------+--------------------+------------+--------+----------+----------+-----+----------+--------+--------+--------+------------+--------------+
|        isin|mnemonic|        securityDesc|securityType|currency|securityID|      date| time|startPrice|maxPrice|minPrice|endPrice|tradedVolume|numberOfTrades|
+------------+--------+--------------------+------------+--------+----------+----------+-----+----------+--------+--------+--------+------------+--------------+
|AT0000A0E9W5|    SANT|         S+T AG O.N.|Common stock|     EUR|   2504159|2022-02-01|08:00|     16.36|   16.49|   16.35|   16.35|      2334.0|             4|
|DE000A0DJ6J9|     S92|SMA SOLAR TECHNOL.AG|Common stock|     EUR|   2504287|2022-02-01|08:00|     31.78|   31.88|    31.6|   31.82|      7672.0|            15|
|DE000A0D6554|    NDX1|      NORDEX SE O.N.|Common stock|     EUR|   2504290|2022-02-01|08:00|     14.19|   14.31|   14.19|   14.31|     57633.0|            55|
|DE000A0D9PT0|     MTX|MTU AERO EN